In [1]:
import sqlite3
import pandas as pd

In [2]:
# Load JSON data in chunks to avoid memory issues
def load_dataset(file_lists, prefix_path, chunk_size=10000):
    df_dict = {}
    for file in file_lists:
        try:
            df_chunks = []
            total_records = 0

            for chunk in pd.read_json(prefix_path + file, lines=True, chunksize=chunk_size):
                df_chunks.append(chunk)
                total_records += chunk.shape[0]

            df = pd.concat(df_chunks, ignore_index=True)
            df_dict[file] = df
            print(f"Total records in {file}: {df.shape[0]}.")

        except Exception as e:
            print(f"Error: {e}")
            continue
    return df_dict

In [3]:
# Convert Timestamps to strings in the dataset
def convert_timestamp_to_str(df, date_columns):
    for col in date_columns:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else None)
    return df

In [4]:
# Insert data into business_details and related tables in batches
def insert_business_data(df_business, conn, batch_size=10000):
    cursor = conn.cursor()
    cursor.execute('BEGIN TRANSACTION')

    for i in range(0, len(df_business), batch_size):
        batch = df_business.iloc[i:i + batch_size]

        # Insert business details
        cursor.executemany('''INSERT OR IGNORE INTO business_details 
                              (business_id, name, address, city, state, postal_code, latitude, longitude, stars, review_count, is_open)
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                           batch[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open']].values.tolist())

        # Insert categories
        for _, row in batch.iterrows():
            categories = row['categories'].split(', ') if row['categories'] else []
            cursor.executemany('INSERT OR IGNORE INTO business_categories (business_id, category) VALUES (?, ?)', 
                               [(row['business_id'], cat) for cat in categories])

    conn.commit()

In [5]:
# Insert review data
def insert_review_data(df_review, conn, batch_size=10000):
    cursor = conn.cursor()
    cursor.execute('BEGIN TRANSACTION')

    # Convert 'date' to string
    df_review = convert_timestamp_to_str(df_review, ['date'])

    for i in range(0, len(df_review), batch_size):
        batch = df_review.iloc[i:i + batch_size]
        cursor.executemany('''INSERT OR IGNORE INTO review_data 
                              (review_id, user_id, business_id, stars, date, text, useful, funny, cool)
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                           batch[['review_id', 'user_id', 'business_id', 'stars', 'date', 'text', 'useful', 'funny', 'cool']].values.tolist())

    conn.commit()

In [6]:
# Insert user data 
def insert_user_data(df_user, conn, batch_size=10000):
    cursor = conn.cursor()
    cursor.execute('BEGIN TRANSACTION')

    # Convert 'yelping_since' to string
    # df_user = convert_timestamp_to_str(df_user, ['yelping_since'])

    for i in range(0, len(df_user), batch_size):
        batch = df_user.iloc[i:i + batch_size]
        cursor.executemany('''INSERT OR IGNORE INTO user_data 
                              (user_id, name, review_count, yelping_since, useful, funny, cool, fans, average_stars)
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                           batch[['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny', 'cool', 'fans', 'average_stars']].values.tolist())

    conn.commit()

In [7]:
# Insert tip data
def insert_tip_data(df_tip, conn, batch_size=10000):
    cursor = conn.cursor()
    cursor.execute('BEGIN TRANSACTION')

    # Convert 'date' to string
    df_tip = convert_timestamp_to_str(df_tip, ['date'])

    for i in range(0, len(df_tip), batch_size):
        batch = df_tip.iloc[i:i + batch_size]
        cursor.executemany('''INSERT OR IGNORE INTO tip_data 
                              (user_id, business_id, text, date, compliment_count)
                              VALUES (?, ?, ?, ?, ?)''',
                           batch[['user_id', 'business_id', 'text', 'date', 'compliment_count']].values.tolist())

    conn.commit()

In [8]:
# Create tables for Business, Review, User, and Tip data
def create_tables(conn):
    cursor = conn.cursor()

    # Create table for business details
    cursor.execute('''CREATE TABLE IF NOT EXISTS business_details (
                        business_id TEXT PRIMARY KEY,
                        name TEXT,
                        address TEXT,
                        city TEXT,
                        state TEXT,
                        postal_code TEXT,
                        latitude REAL,
                        longitude REAL,
                        stars REAL,
                        review_count INTEGER,
                        is_open INTEGER
                    )''')

    # Create table for business categories
    cursor.execute('''CREATE TABLE IF NOT EXISTS business_categories (
                        business_id TEXT,
                        category TEXT,
                        FOREIGN KEY (business_id) REFERENCES business_details(business_id)
                    )''')

    # Create table for reviews
    cursor.execute('''CREATE TABLE IF NOT EXISTS review_data (
                        review_id TEXT PRIMARY KEY,
                        user_id TEXT,
                        business_id TEXT,
                        stars INTEGER,
                        date TEXT,
                        text TEXT,
                        useful INTEGER,
                        funny INTEGER,
                        cool INTEGER,
                        FOREIGN KEY (business_id) REFERENCES business_details(business_id),
                        FOREIGN KEY (user_id) REFERENCES user_data(user_id)
                    )''')

    # Create table for users
    cursor.execute('''CREATE TABLE IF NOT EXISTS user_data (
                        user_id TEXT PRIMARY KEY,
                        name TEXT,
                        review_count INTEGER,
                        yelping_since TEXT,
                        useful INTEGER,
                        funny INTEGER,
                        cool INTEGER,
                        fans INTEGER,
                        average_stars REAL
                    )''')

    # Create table for tips
    cursor.execute('''CREATE TABLE IF NOT EXISTS tip_data (
                        user_id TEXT,
                        business_id TEXT,
                        text TEXT,
                        date TEXT,
                        compliment_count INTEGER,
                        FOREIGN KEY (business_id) REFERENCES business_details(business_id),
                        FOREIGN KEY (user_id) REFERENCES user_data(user_id)
                    )''')

    conn.commit()

In [9]:
# Main execution
folder_path = '../../data/'
prefix_path = folder_path + 'raw_datasets/yelp/'
file_list = [
    "sampled_yelp_academic_dataset_business.json",
    "sampled_yelp_academic_dataset_review.json",
    "sampled_yelp_academic_dataset_user.json",
    "sampled_yelp_academic_dataset_tip.json"
]

# Load the datasets in chunks
df_dict = load_dataset(file_list, prefix_path)

# Split the datasets
df_business = df_dict["sampled_yelp_academic_dataset_business.json"]
df_review = df_dict["sampled_yelp_academic_dataset_review.json"]
df_user = df_dict["sampled_yelp_academic_dataset_user.json"]
df_tip = df_dict["sampled_yelp_academic_dataset_tip.json"]

# Create connections for separate database files
db_path_business = '../../data/processed_data/yelp_data/yelp_business_data.db'
db_path_review = '../../data/processed_data/yelp_data/yelp_review_data.db'
db_path_user = '../../data/processed_data/yelp_data/yelp_user_data.db'
db_path_tip = '../../data/processed_data/yelp_data/yelp_tip_data.db'

Total records in sampled_yelp_academic_dataset_business.json: 78059.
Total records in sampled_yelp_academic_dataset_review.json: 980418.
Total records in sampled_yelp_academic_dataset_user.json: 229447.
Total records in sampled_yelp_academic_dataset_tip.json: 173085.


In [18]:
print(df_business['categories'][0])
print()
print(df_business['categories'][1])
print()
print(df_business['categories'][2])

Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores

Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries

Department Stores, Shopping, Fashion


In [ ]:
conn_business = sqlite3.connect(db_path_business)
conn_review = sqlite3.connect(db_path_review)
conn_user = sqlite3.connect(db_path_user)
conn_tip = sqlite3.connect(db_path_tip)

# Create tables
create_tables(conn_business)
create_tables(conn_review)
create_tables(conn_user)
create_tables(conn_tip)

# Insert data in batches
insert_business_data(df_business, conn_business)
insert_review_data(df_review, conn_review)
insert_user_data(df_user, conn_user)
insert_tip_data(df_tip, conn_tip)

# Close connections
conn_business.close()
conn_review.close()
conn_user.close()
conn_tip.close()

print("Data has been successfully stored in the databases.")